In [1]:
import os
import random
import numpy as np
import errno
import csv
from PIL import Image
import cv2
import matplotlib.pyplot as plt
from shapely.geometry import Polygon
import json

In [2]:
def get_bbox(geom):
    pairs = []

    for i in range(0, len(geom)-1, 2):
        pairs.append((geom[i], geom[i+1]))

    x = []
    y = []
    for i in pairs:
        x.append(i[0])
        y.append(i[1])

    bbox = [min(x), min(y), round(max(x)-min(x),2), round(max(y)-min(y),2)]
    
    return bbox

In [3]:
info = {
        "description": "Felipe Sa 2021 - HLB",
        "url": "http://siteaqui.com",
        "version": "1.0",
        "year": 2021,
        "contributor": "Felipe Sa",
        "date_created": "2021/01/01"
        }

licenses = [
                {
                "url": "http://creativecommons.org/licenses/by-nc-sa/2.0/",
                "id": 1,
                "name": "Attribution-NonCommercial-ShareAlike License"
                },
                {
                "url": "http://creativecommons.org/licenses/by-nc/2.0/",
                "id": 2,
                "name": "Attribution-NonCommercial License"
                }
            ]

categories = [{"supercategory": "canopy","id": 1,"name": "hamlin"}]

In [4]:
def compose_images(foreground_path, background_path):
    # Make sure the background path is valid and open the image
    assert os.path.exists(background_path), 'image path does not exist: {}'.format(background_path)
    assert os.path.splitext(background_path)[1].lower() in ['.png', '.jpg', '.jpeg'], \
        'foreground must be a .png or .jpg file: {}'.format(foreground_path)
    background = Image.open(background_path)
    background = background.convert('RGBA')
    
    # Make sure the foreground path is valid and open the image
    assert os.path.exists(foreground_path), 'image path does not exist: {}'.format(foreground_path)
    foreground = Image.open(foreground_path)
    foreground_alpha = np.array(foreground.getchannel(3))
    assert np.any(foreground_alpha == 0), 'foreground needs to have some transparency: {}'.format(foreground_path)
    
    # Rotate the foreground
    angle_degrees = random.randint(0, 359)
    foreground = foreground.rotate(angle_degrees, resample=Image.BICUBIC, expand=True)
    
    # Scale the foreground
    scale = random.random() * .5 + .5 # Pick something between .5 and 1
    new_size = (int(foreground.size[0] * scale), int(foreground.size[1] * scale))
    foreground = foreground.resize(new_size, resample=Image.BICUBIC)
    
    # Add any other transformations here...
    
    # Choose a random x,y position for the foreground
    max_xy_position = (background.size[0] - foreground.size[0], background.size[1] - foreground.size[1])
    assert max_xy_position[0] >= 0 and max_xy_position[1] >= 0, \
        'foreground {} is to big for the background {}'.format(foreground_path, background_path)
    paste_position = (random.randint(0, max_xy_position[0]), random.randint(0, max_xy_position[1]))
    
    # Create a new foreground image as large as the background and paste it on top
    new_foreground = Image.new('RGBA', background.size, color = (0, 0, 0, 0))
    new_foreground.paste(foreground, paste_position)
        
    # Extract the alpha channel from the foreground and paste it into a new image the size of the background
    alpha_mask = foreground.getchannel(3)
    new_alpha_mask = Image.new('L', background.size, color=0)
    new_alpha_mask.paste(alpha_mask, paste_position)
    
    composite = Image.composite(new_foreground, background, new_alpha_mask)
    
    # Grab the alpha pixels above a specified threshold
    alpha_threshold = 200
    mask_arr = np.array(np.greater(np.array(new_alpha_mask), alpha_threshold), dtype=np.uint8)
    hard_mask = Image.fromarray(np.uint8(mask_arr) * 255, 'L')
    
    # Get annotations
    mask_arr = np.stack([new_alpha_mask,new_alpha_mask,new_alpha_mask], axis=2)
    gray_image = cv2.cvtColor(mask_arr, cv2.COLOR_BGR2GRAY)
    ret, thresh = cv2.threshold(gray_image, 50, 255, cv2.THRESH_BINARY)
    im, contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    
    x_coords = []
    y_coords = []
    annotations = []
    for i in contours[0]:
        annotations.append(int(i[0][0]))
        x_coords.append(int(i[0][0]))
        
        annotations.append(int(i[0][1]))
        y_coords.append(int(i[0][1]))
        
    area = Polygon(zip(x_coords, y_coords)).area

    return composite, annotations, area

In [5]:
# Get lists of foreground and background image paths
dataset_dir = '../IMGS/synthetic_dataset'
backgrounds_dir = os.path.join(dataset_dir, 'backgrounds', 'JPG')
foregrounds_dir = os.path.join(dataset_dir, 'foregrounds', 'PNG')
backgrounds = [os.path.join(backgrounds_dir, file_name) for file_name in os.listdir(backgrounds_dir)]
foregrounds = [os.path.join(foregrounds_dir, file_name) for file_name in os.listdir(foregrounds_dir)]

# Create an output directory
output_dir = os.path.join(dataset_dir, 'generated')
try:
    os.mkdir(output_dir)
except OSError as exc:
    if exc.errno != errno.EEXIST:
        raise
    pass

# Create a list to keep track of images and annotations
images = []
annotations = []

i = 1
fore_id = 1

# Generate 5 new images
while i <= 1:
    background_path = random.choice(backgrounds)
    foreground_path = random.choice(foregrounds)

    composite, geom_px, area = compose_images(foreground_path, background_path)
    
#     composite_path = os.path.join(output_dir, '{0:04}.jpg'.format(i))
#     composite.convert('RGB').save(composite_path)
    
    images.append({ "license": 1,
                    "file_name": "{:04}.jpg".format(i),
                    "coco_url": "empty",
                    "height": composite.size[0],
                    "width": composite.size[1],
                    "date_captured": "2020-01-01 00:00:00",
                    "flickr_url": "empty",
                    "id": i
                  })
    
    annotations.append({"segmentation": [geom_px],
                        "area": area, # Atenção aqui!! 0.0025 é para o tamanho do pixel de 5cm (0,05 x 0,05 = 0,0025)
                        "iscrowd": 0,
                        "image_id": i,
                        "bbox": get_bbox(geom_px),
                        "category_id": 1,
                        "id": i # Não definido ainda
                       })
    
    while fore_id < 7:
        foreground_path = random.choice(foregrounds)

        composite, geom_px, area = compose_images(foreground_path, composite)



        annotations.append({"segmentation": [geom_px],
                            "area": area, # Atenção aqui!! 0.0025 é para o tamanho do pixel de 5cm (0,05 x 0,05 = 0,0025)
                            "iscrowd": 0,
                            "image_id": i,
                            "bbox": get_bbox(geom_px),
                            "category_id": 1,
                            "id": i # Não definido ainda
                           })
        fore_id += 1
    
    i += 1
    
composite_path = os.path.join(output_dir, '{0:04}.jpg'.format(i))
composite.convert('RGB').save(composite_path)
    
annotation = {"info": info, 
              "licenses": licenses, 
              "images": images, 
              "categories": categories, 
              "annotations": annotations}


with open(os.path.join(dataset_dir, 'coco_annotations.json'), 'w') as outfile:
    json.dump(annotation, outfile)

TypeError: stat: path should be string, bytes, os.PathLike or integer, not Image